## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vierzon,FR,47.2101,2.0625,64.99,56,80,5.46,broken clouds
1,1,Nyaunglebin,MM,17.9500,96.7333,75.72,95,100,4.45,overcast clouds
2,2,Kimbe,PG,-5.5502,150.1429,74.59,92,99,3.11,overcast clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,59.94,43,100,9.53,overcast clouds
4,4,Mnogovershinnyy,RU,53.9353,139.9242,49.06,82,100,2.33,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like to enjoy?"))
max_temp=float(input("What is the maximum temperature you would like to enjoy?"))

What is the minimum temperature you would like to enjoy?75
What is the maximum temperature you would like to enjoy?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & \
                                 (city_data_df["Max Temp"]>= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Nyaunglebin,MM,17.9500,96.7333,75.72,95,100,4.45,overcast clouds
7,7,Albany,US,42.6001,-73.9662,89.17,47,63,8.01,broken clouds
10,10,Puerto Ayora,EC,-0.7393,-90.3518,80.56,81,92,10.18,overcast clouds
12,12,Luganville,VU,-15.5333,167.1667,79.30,84,39,4.92,scattered clouds
13,13,Piacabucu,BR,-10.4056,-36.4344,77.13,89,100,11.25,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Nyaunglebin,MM,17.9500,96.7333,75.72,95,100,4.45,overcast clouds
7,7,Albany,US,42.6001,-73.9662,89.17,47,63,8.01,broken clouds
10,10,Puerto Ayora,EC,-0.7393,-90.3518,80.56,81,92,10.18,overcast clouds
12,12,Luganville,VU,-15.5333,167.1667,79.30,84,39,4.92,scattered clouds
13,13,Piacabucu,BR,-10.4056,-36.4344,77.13,89,100,11.25,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
702,702,Prata,BR,-19.3072,-48.9242,80.80,39,0,5.57,clear sky
705,705,Dubai,AE,25.2582,55.3047,86.25,65,0,6.91,clear sky
707,707,Miranda,CO,3.2499,-76.2281,77.13,90,100,1.70,overcast clouds
709,709,Garissa,KE,-0.4536,39.6401,76.28,76,34,16.40,scattered clouds


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Nyaunglebin,MM,75.72,overcast clouds,17.9500,96.7333,
7,Albany,US,89.17,broken clouds,42.6001,-73.9662,
10,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,
12,Luganville,VU,79.30,scattered clouds,-15.5333,167.1667,
13,Piacabucu,BR,77.13,overcast clouds,-10.4056,-36.4344,
14,Kapaa,US,79.48,broken clouds,22.0752,-159.3190,
17,Korla,CN,79.75,overcast clouds,41.7597,86.1469,
18,Saint George,US,76.62,clear sky,37.1041,-113.5841,
21,Jamestown,US,87.53,clear sky,42.0970,-79.2353,
25,Atuona,PF,77.88,light rain,-9.8000,-139.0333,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel=hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Nyaunglebin,MM,75.72,overcast clouds,17.9500,96.7333,Thi ha
10,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
12,Luganville,VU,79.30,scattered clouds,-15.5333,167.1667,The Espiritu Hotel
13,Piacabucu,BR,77.13,overcast clouds,-10.4056,-36.4344,Piaçabuçu News
14,Kapaa,US,79.48,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
702,Prata,BR,80.80,clear sky,-19.3072,-48.9242,Hotel Pontal Aluga-se Quitinetes e suítes mens...
705,Dubai,AE,86.25,clear sky,25.2582,55.3047,"Radisson Blu Hotel, Dubai Deira Creek"
707,Miranda,CO,77.13,overcast clouds,3.2499,-76.2281,unidiversidad organica la utopia
709,Garissa,KE,76.28,scattered clouds,-0.4536,39.6401,Sharif Plaza Hotel


In [19]:
# 8a. Create the output File (CSV)
output_data_file="../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Decription</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))